In [1]:
from zipline.api import order_target_percent, record, symbol, schedule_function, date_rules, order
from zipline.api import set_commission, commission, set_slippage, slippage
from zipline import run_algorithm
from zipline.utils.events import time_rules

import pandas as pd
import numpy as np

import requests
from datetime import datetime
from zipline.api import set_commission, set_slippage
from zipline.finance.commission import PerShare, PerDollar
from zipline.finance.slippage import VolumeShareSlippage

In [2]:
RETS_FILE_PATH = 'historical_returns.csv'

In [3]:
def get_weights_for_date(date):
  
    # The base URL of your Flask application
    base_url = "http://localhost:5000/get_weights"
    
    # Prepare the query parameters
    params = {'date': str(date)}
    
    try:
        # Sending a GET request to the endpoint
        response = requests.get(base_url, params=params)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            return data.get('weights')
        else:
            # Handle possible errors (e.g., invalid date format, missing date parameter)
            print(f"Error: {response.json()['error']}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [4]:
# Define the initialize function
def initialize(context):

        # Set the commission model
    #set_commission(PerDollar(cost=0.0001))
    set_commission(PerDollar(cost=0))
    
    # Set the slippage model
    # VolumeShareSlippage takes 'volume_limit' and 'price_impact' as parameters
    # 'volume_limit' is the fraction of a bar's volume that your order can take up
    # 'price_impact' is the impact your order has on the price per share for each share traded
    #set_slippage(VolumeShareSlippage(volume_limit=0.025, price_impact=0.01))
    
    # Slippage model: Assuming 0.05% slippage per trade
    #set_slippage(slippage.VolumeShareSlippage(volume_limit=0.025, price_impact=0.0005))

    
    context.tickers = ['VWO', 'SPY', 'VNQ', 'LQD', 'DBC']
    context.assets = [symbol(ticker) for ticker in context.tickers]
    
    # Schedule the rebalance function to run every Monday at market open
    schedule_function(rebalance, date_rules.week_start(), time_rules.market_open())
    # Set the context attributes needed
    context.lookback_days = 50 # look back over 50 days for the moving average

# Define the rebalance function
def rebalance(context, data):
    #print(context.get_datetime().date())
     
    # Get historical data for the assets
    historical_data = data.history(context.assets, "price", context.lookback_days + 1, "1d")
    
#     # Calculate daily returns
    daily_returns = historical_data.pct_change().dropna()
    daily_returns.to_csv(RETS_FILE_PATH)
    current_date = context.get_datetime().date()
    
    print(f'Handling {current_date}')
    
    weights = get_weights_for_date(current_date)
    print(weights)
    
    # Place orders
    for asset, weight in zip(context.assets, weights):
        if data.can_trade(asset):
            order_target_percent(asset, weight)
            #print(f'ordered {asset}: {weight}')
    
    # Record some metrics
    record(weights=weights)

# Define the analyze function (optional)
def analyze(context, perf):
    # Plot portfolio weights over time
    #perf.weights.plot()
    pass

In [11]:
old_dates  = [
    (pd.Timestamp('2017-12-15', tz='UTC'), pd.Timestamp('2018-12-15', tz='UTC')),
    (pd.Timestamp('2018-12-15', tz='UTC'), pd.Timestamp('2019-12-15', tz='UTC')),
    (pd.Timestamp('2019-12-15', tz='UTC'), pd.Timestamp('2020-12-15', tz='UTC')),
    (pd.Timestamp('2017-12-15', tz='UTC'), pd.Timestamp('2020-12-15', tz='UTC')),
] 


new_dates  = [
    (pd.Timestamp('2020-12-15', tz='UTC'), pd.Timestamp('2021-12-15', tz='UTC')),
    (pd.Timestamp('2021-12-15', tz='UTC'), pd.Timestamp('2022-12-15', tz='UTC')),
    (pd.Timestamp('2022-12-15', tz='UTC'), pd.Timestamp('2023-12-15', tz='UTC')),
    (pd.Timestamp('2020-12-15', tz='UTC'), pd.Timestamp('2023-12-15', tz='UTC')),
] 

In [12]:
dates = old_dates
dates = new_dates

In [13]:
perfs = []
capital_base = 1e7

for ds in dates:
    start_date, end_date = ds
    print(f'\n*********\n   ({start_date.date()}, {end_date.date()}) \n*********\n')
    perf = run_algorithm(
        start=start_date,
        end=end_date,
        initialize=initialize,
        analyze=analyze,
        capital_base=capital_base,
        data_frequency='daily',
        bundle='thesis1' 
    )
    perfs.append(perf)


*********
   (2020-12-15, 2021-12-15) 
*********



C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(


Handling 2020-12-21
[-0.3147144913673401, 0.9999938607215881, -0.19605451822280884, 0.48103758692741394, 0.029737472534179688]
Handling 2020-12-28


C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(


[0.06423242390155792, 0.201311394572258, -0.22191432118415833, 0.9754660129547119, -0.019095532596111298]
Handling 2021-01-04
[0.18179692327976227, 0.1357293576002121, 0.33061423897743225, 0.798307478427887, -0.44644805788993835]
Handling 2021-01-11
[-0.16662244498729706, 0.09917405247688293, 0.05370962247252464, 0.8366597294807434, 0.17707905173301697]
Handling 2021-01-19
[-0.1374533474445343, 0.22342228889465332, 0.1632424145936966, 0.9999035000801086, -0.24911491572856903]
Handling 2021-01-25
[-0.25585636496543884, 0.2912152111530304, 0.2894293963909149, 0.5518965125083923, 0.12331523001194]
Handling 2021-02-01
[-0.2723153829574585, 0.3272469937801361, 0.029797865077853203, 0.5646688342094421, 0.3506016731262207]
Handling 2021-02-08
[-0.11866391450166702, 0.5451988577842712, 0.06536330282688141, 0.24601788818836212, 0.26208382844924927]
Handling 2021-02-16
[-0.12957122921943665, 0.7911673188209534, 0.09300418943166733, 0.5898680090904236, -0.34446826577186584]
Handling 2021-02-22
[-

...

Handling 2020-12-21
[-0.3147144913673401, 0.9999938607215881, -0.19605451822280884, 0.48103758692741394, 0.029737472534179688]
Handling 2020-12-28


C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\seidm\miniconda3\envs\ml4t\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)


[0.06423242390155792, 0.201311394572258, -0.22191432118415833, 0.9754660129547119, -0.019095532596111298]
Handling 2021-01-04
[0.18179692327976227, 0.1357293576002121, 0.33061423897743225, 0.798307478427887, -0.44644805788993835]
Handling 2021-01-11
[-0.16662244498729706, 0.09917405247688293, 0.05370962247252464, 0.8366597294807434, 0.17707905173301697]
Handling 2021-01-19
[-0.1374533474445343, 0.22342228889465332, 0.1632424145936966, 0.9999035000801086, -0.24911491572856903]
Handling 2021-01-25
[-0.25585636496543884, 0.2912152111530304, 0.2894293963909149, 0.5518965125083923, 0.12331523001194]
Handling 2021-02-01
[-0.2723153829574585, 0.3272469937801361, 0.029797865077853203, 0.5646688342094421, 0.3506016731262207]
Handling 2021-02-08
[-0.11866391450166702, 0.5451988577842712, 0.06536330282688141, 0.24601788818836212, 0.26208382844924927]
Handling 2021-02-16
[-0.12957122921943665, 0.7911673188209534, 0.09300418943166733, 0.5898680090904236, -0.34446826577186584]
Handling 2021-02-22
[-

[0.2264726310968399, 0.2935928404331207, -0.20944082736968994, 0.45697829127311707, 0.23239704966545105]
Handling 2022-04-11
[-0.2603681683540344, 0.3650159537792206, 0.18843507766723633, 0.4251178205013275, 0.28179931640625]
Handling 2022-04-18
[0.34715554118156433, 0.09014516323804855, 0.06449677795171738, 0.665463387966156, -0.16726087033748627]
Handling 2022-04-25
[-0.24926406145095825, 0.07044554501771927, 0.17834961414337158, 1.0000001192092896, 0.00046881416346877813]
Handling 2022-05-02
[0.04894980415701866, 0.5167993307113647, -0.3685152530670166, 0.45514991879463196, 0.3476162254810333]
Handling 2022-05-09
[-0.2767837941646576, 0.24197588860988617, 0.2980041801929474, 0.7937045693397522, -0.05690087378025055]
Handling 2022-05-16
[-0.20336313545703888, 0.6164143085479736, -0.10514765977859497, 0.36957862973213196, 0.3225177526473999]
Handling 2022-05-23
[-0.3105522394180298, 0.15926922857761383, 0.17046262323856354, 0.7877179384231567, 0.1931024193763733]
Handling 2022-05-31
[

[-0.35585376620292664, 0.5756813287734985, 0.5550070405006409, 0.2765748202800751, -0.051409561187028885]
Handling 2023-07-17
[0.200853168964386, 0.5329563021659851, 0.1091352179646492, -0.8429514765739441, 1.0000066757202148]
Handling 2023-07-24
[-0.34380027651786804, 0.8791989088058472, 0.21739238500595093, -0.059084195643663406, 0.3062930703163147]
Handling 2023-07-31
[-0.10057907551527023, 0.929876446723938, 0.4039629399776459, -0.49029701948165894, 0.2570367753505707]
Handling 2023-08-07
[-0.40870949625968933, 0.9999648332595825, 0.1481720507144928, -0.09260222315788269, 0.35317471623420715]
Handling 2023-08-14
[-0.13973264396190643, 0.4037521779537201, -0.30205294489860535, 0.4144047200679779, 0.6236287355422974]
Handling 2023-08-21
[-0.25525352358818054, 0.42890533804893494, 0.0647810846567154, 0.5591596364974976, 0.20240743458271027]
Handling 2023-08-28
[-0.2170003205537796, 0.4464821517467499, -0.07123948633670807, 0.334300696849823, 0.50745689868927]
Handling 2023-09-05
[-0.1

In [9]:
## Sharpe Ratio: 5.319768807195642

import empyrical as ep

sharpes = []

for perf in perfs:
#     cumulative_returns = ep.cum_returns_final(perf['returns'])
#     annualized_returns = ep.annual_return(perf['returns'])
    sharpe_ratio = ep.sharpe_ratio(perf['returns'])
    #max_drawdown = ep.max_drawdown(perf['returns'])
    sharpes.append(sharpe_ratio)

In [10]:
sharpes

[1.9621940031877272,
 -1.036449722463757,
 0.5349122637473209,
 0.18931129862080595]

## Save the result

In [12]:
len(perfs)

4

In [13]:
PERF_NAME = 'dense_full_opti'
perfs[3].to_csv(fr'..\analytics\perfs\old\{PERF_NAME}_perf.csv')

# Analytics

In [ ]:
# mean_returns: -0.00022047444443614898
# std: 0.006793907584478022
# Cumulative Returns: -0.1673630146755024
# Annualized Returns: -0.05907459961792738
# Sharpe Ratio: -0.5131137893372564
# Max Drawdown: -0.2874978151290583

In [9]:
## Sharpe Ratio: 5.319768807195642

import empyrical as ep

sharpes = []

for perf in perfs:
#     cumulative_returns = ep.cum_returns_final(perf['returns'])
#     annualized_returns = ep.annual_return(perf['returns'])
    sharpe_ratio = ep.sharpe_ratio(perf['returns'])
    #max_drawdown = ep.max_drawdown(perf['returns'])
    sharpes.append(sharpe_ratio)

In [ ]:
tmp = [perf['weights'][i][1] for i in range(4, len(perf['weights']))]
tmp = np.array(tmp)
tmp[tmp > 1]

In [ ]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

perf['cumulative_returns'] = (1 + perf['returns']).cumprod() - 1

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(perf['cumulative_returns'], label='Cumulative Returns')
plt.title('Cumulative Returns Over Time')
plt.xlabel('Time')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.show()

In [ ]:
perf['weights']

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

# Define the dates
start_date = datetime(2007, 1, 3)
retraining_dates = [datetime(2021, 12, 15), datetime(2022, 12, 15)]
test_start_date = datetime(2021, 12, 15)
end_date = datetime(2022, 12, 15)

# Plotting
fig, ax = plt.subplots(figsize=(10, 2))

# Highlight the training period
ax.plot([start_date, test_start_date], [1, 1], color="skyblue", linewidth=8, label='Training Period')

# Highlight the testing period
ax.plot([test_start_date, end_date], [1, 1], color="lightgreen", linewidth=8, label='Testing Period')

# Mark the retraining dates
for date in retraining_dates:
    plt.plot([date, date], [0.8, 1.2], color="red", linestyle='--', linewidth=2)

# Adding annotations for retraining dates
for date in retraining_dates:
    ax.text(date, 1.25, '', ha='center', color='red')

# Adjust the plot
ax.set_ylim(0.75, 1.5)
ax.yaxis.set_visible(False)  # Hide the y-axis
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Adding labels and title
plt.title('Timeline of Training, Testing, and Model Retraining')

# Adding custom legend
handles, labels = ax.get_legend_handles_labels()
custom_lines = [plt.Line2D([0], [0], color="skyblue", lw=8),
                plt.Line2D([0], [0], color="lightgreen", lw=8),
                plt.Line2D([0], [0], color="red", linestyle='--', lw=2)]
plt.legend(custom_lines, ['Training Period', 'Testing Period', 'Retraining'])

plt.tight_layout()
plt.show()
